In [1]:
import os
import json
import glob
import random
import collections

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers



TYPES = ["FLAIR", "T1w", "T2w", "T1wCE"]
WHITE_THRESHOLD = 10 # out of 255
EXCLUDE = [109, 123, 709]


train_df = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
test_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]
def load_dicom(path, size = 224):
    ''' 
    Reads a DICOM image, standardizes so that the pixel values are between 0 and 1, then rescales to 0 and 255
    
    Note super sure if this kind of scaling is appropriate, but everyone seems to do it. 
    '''
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in TYPES)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]
IMAGE_SIZE = 128

def get_all_data_for_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', IMAGE_SIZE)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_for_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', IMAGE_SIZE)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)
X, y, trainidt = get_all_data_for_train('T1wCE')
X_test, testidt = get_all_data_for_test('T1wCE')
X.shape, y.shape, trainidt.shape

2021-11-16 09:04:34.227676: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

((16196, 128, 128), (16196,), (16196,))

In [2]:
X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = train_test_split(X, y, trainidt, test_size=0.2, random_state=40)

split = int(X.shape[0] * 0.8)

# X_train = X[:split]
# X_valid = X[split:]

# y_train = y[:split]
# y_valid = y[split:]

# trainidt_train = trainidt[:split]
# trainidt_valid = trainidt[split:]

X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, trainidt_train.shape, trainidt_valid.shape

2021-11-16 09:11:38.042926: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-16 09:11:38.046520: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-16 09:11:38.102924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-16 09:11:38.103651: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-11-16 09:11:38.103719: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-16 09:11:38.146089: I tensorflow/stream_executor/platform/def

(TensorShape([12956, 128, 128, 1]),
 (12956, 2),
 TensorShape([3240, 128, 128, 1]),
 (3240, 2),
 (12956,),
 (3240,))

In [3]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [4]:
np.random.seed(0)
random.seed(12)
tf.random.set_seed(12)

inpt = keras.Input(shape=(128, 128, 1))

h = keras.layers.experimental.preprocessing.Rescaling(1./255)(inpt)
# h = data_augmentation(h)

# convolutional layer!
h = keras.layers.Conv2D(32, kernel_size=(3, 3),strides=(1,1), activation="relu", name="Conv_1", padding="valid")(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(64, kernel_size=(3, 3),strides=(1,1), activation="relu", name="Conv_2", padding="same")(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# pooling layer
h = keras.layers.MaxPool2D(pool_size=(2,2))(h) 
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# convolutional layer!
h = keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu", name="Conv_3",padding ="same")(h)
# h = tf.keras.layers.BatchNormalization(axis=-1)(h)
# pooling layer
# h = keras.layers.MaxPool2D(pool_size=(1,1))(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", name="Conv_4",padding ="valid")(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", name="Conv_5",padding ="same")(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.MaxPool2D(pool_size=(2,2))(h)
h = tf.keras.layers.BatchNormalization(axis=-1)(h)
h = keras.layers.Dropout(0.3)(h)   

h = keras.layers.Flatten()(h) 
# h = global_average_layer(h)
h = keras.layers.Dense(128, activation='relu')(h)   

output = keras.layers.Dense(2, activation="softmax")(h)

model = keras.Model(inpt, output)

from tensorflow.keras.optimizers import SGD
# opt = SGD(lr=0.1)

checkpoint_filepath = 'best_model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_filepath,
save_weights_only=False,
monitor='val_auc',
mode='max',
save_best_only=True,
save_freq='epoch')

model.compile(loss='categorical_crossentropy',
             optimizer=tf.keras.optimizers.SGD(learning_rate =0.0001),
             metrics=[tf.keras.metrics.AUC()])

history = model.fit(x=X_train, y = y_train, epochs=50, callbacks=[model_checkpoint_callback], validation_data= (X_valid, y_valid))

2021-11-16 09:12:00.010772: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-16 09:12:00.022875: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000129999 Hz


Epoch 1/50


2021-11-16 09:12:00.906857: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-11-16 09:12:01.751464: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-11-16 09:12:01.773666: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


405/405 [==============================] - 28s 50ms/step - loss: 0.8104 - auc: 0.5507 - val_loss: 0.7151 - val_auc: 0.5854
Epoch 2/50
405/405 [==============================] - 19s 46ms/step - loss: 0.6862 - auc: 0.6180 - val_loss: 0.6608 - val_auc: 0.6446
Epoch 3/50
405/405 [==============================] - 19s 46ms/step - loss: 0.6489 - auc: 0.6662 - val_loss: 0.6491 - val_auc: 0.6691
Epoch 4/50
405/405 [==============================] - 19s 46ms/step - loss: 0.6306 - auc: 0.6960 - val_loss: 0.6385 - val_auc: 0.6851
Epoch 5/50
405/405 [==============================] - 19s 46ms/step - loss: 0.6187 - auc: 0.7107 - val_loss: 0.6273 - val_auc: 0.7037
Epoch 6/50
405/405 [==============================] - 19s 46ms/step - loss: 0.5923 - auc: 0.7449 - val_loss: 0.6296 - val_auc: 0.7040
Epoch 7/50
405/405 [==============================] - 19s 46ms/step - loss: 0.5796 - auc: 0.7631 - val_loss: 0.6088 - val_auc: 0.7296
Epoch 8/50
405/405 [==============================] - 19s 46ms/step - los

In [5]:
file_path = '../input/fork-of-rsna-miccai-2dcnn-training/best_model_inception.h5'

In [6]:
def loss(y_true, y_pred):
    return - (1 - theta(y_true - margin) * theta(y_pred - margin)
                - theta(1 - margin - y_true) * theta(1 - margin - y_pred)
             ) * (y_true * K.log(y_pred + 1e-8) + (1 - y_true) * K.log(1 - y_pred + 1e-8))

import tensorflow as tf

margin = 0.6
theta = lambda t: (K.sign(t)+1.)/2.

def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x, inputs)
    return x

#model_best = tf.keras.models.load_model(filepath=file_path,custom_objects={"loss":loss,'leaky_relu': tf.nn.leaky_relu,"mish":mish})
# model_best2 = tf.keras.models.load_model('../input/k/juliojuse/k/juliojuse/rsna-miccai-2dcnn-training/best_model.h5')
# model_best3 = tf.keras.models.load_model('../input/d/juliojuse/2dcnn/best_model.h5')

In [9]:
# model_path =[
#  '../input/cnn2d-model/best_model0.747.h5',
#  '../input/cnn2d-model/best_model0.741.h5',
#  '../input/cnn2d-model/best_model0.739.h5',
# '../input/cnn2d-model/best_model0.725.h5',]
model_path = [
 './best_model.h5',
]
pred_list = []
for path in model_path:
    model_best = tf.keras.models.load_model(filepath=path,custom_objects={"loss":loss,'leaky_relu': tf.nn.leaky_relu,'mish':mish})
    sample = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
    y_pred = model_best.predict(X_test)
    #pred = np.argmax(y_pred, axis=1)
    pred_list.append(y_pred)

In [10]:
model_best.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 128, 128, 1)       0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
Conv_2 (Conv2D)              (None, 126, 126, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0     

In [11]:
sample = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

y_pred = sum(pred_list)/len(pred_list)#model_best.predict(X_test)
# y_pred2 = model_best2.predict(X_test)
# y_pred3 = model_best3.predict(X_test)

pred = np.argmax(y_pred, axis=1)
# pred2 = np.argmax(y_pred2, axis=1)
# pred3 = np.argmax(y_pred3, axis=1)

result=pd.DataFrame(testidt)
result[1]=pred #(pred+pred2+pred3)/3

result.columns=['BraTS21ID','MGMT_value']
result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample['BraTS21ID']
# result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'][result2['MGMT_value']>0.8] = 1
# result2['MGMT_value'][result2['MGMT_value']<0.2] = 0
result2.to_csv('submission.csv',index=False)
result2

,BraTS21ID,MGMT_value
0,1,1.000000
1,13,0.772727
2,15,0.941176
3,27,0.725490
4,37,0.909091
...,...,...
82,826,0.285714
83,829,0.800000
84,833,0.312500
85,997,0.166667
